In [89]:
# Ragas 数据集格式要求  ['question', 'answer', 'contexts', 'ground_truths']
'''
{
    "question": [], <-- 问题基于Context的
    "answer": [], <-- 答案基于LLM生成的
    "contexts": [], <-- context
    "ground_truths": [] <-- 标准答案
}
'''

from datasets import Dataset
import os
import json

questions = []
ground_truths = []
answers = []
# contexts = []

llm_name='Mistral-CVE'
# GPT-4o-mini Claude-3-5-Haiku-20241022 Gemini-2.0-Flash-Exp Llama-3-70B Mistral-Medium

type='DPO'
# no_prompt prompt_only
# DPO Llama-3-CVE Mistral-CVE
with open("./evaluation/question.txt",'r',encoding='utf-8') as file:
    questions = [line.strip() for index, line in enumerate(file.readlines()) if index % 10 == 0]
    # questions = [file.readline().strip()]
print(questions)

with open("./evaluation/ground_truths.txt",'r',encoding='utf-8') as file:
    ground_truths = [line.strip() for index, line in enumerate(file.readlines()) if index % 10 == 0]
    # ground_truths=[file.readline().strip()]
print(ground_truths)    
# 读取JSON文件
with open(type+'.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取所有GPT-4o-mini的值
answers = [entry[llm_name] for entry in data if llm_name in entry][::10]
print(answers)

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    # "contexts": contexts,
    "reference": ground_truths
}
dataset = Dataset.from_dict(data)


['What is CVE-2013-3900', 'What is CVE-2019-1133', 'What is CVE-2019-1155', 'What is CVE-2019-1174', 'What is CVE-2019-1190', 'What is CVE-2019-19298', 'What is CVE-2020-0761', 'What is CVE-2020-0908', 'What is CVE-2020-1039', 'What is CVE-2020-1167', 'What is CVE-2020-1252', 'What is CVE-2020-1345', 'What is CVE-2020-1464', 'What is CVE-2020-1486', 'What is CVE-2020-1512', 'What is CVE-2020-1523', 'What is CVE-2020-1533', 'What is CVE-2020-1543', 'What is CVE-2020-1558', 'What is CVE-2020-1583', 'What is CVE-2020-16863', 'What is CVE-2020-16891', 'What is CVE-2020-16908', 'What is CVE-2020-16924', 'What is CVE-2020-16936', 'What is CVE-2020-16955', 'What is CVE-2020-16968', 'What is CVE-2020-16984', 'What is CVE-2020-17016', 'What is CVE-2020-17033', 'What is CVE-2020-17074', 'What is CVE-2020-17109', 'What is CVE-2020-24682', 'What is CVE-2021-21551', 'What is CVE-2021-31890', 'What is CVE-2021-36821', 'What is CVE-2021-45036', 'What is CVE-2022-2513', 'What is CVE-2022-31217', 'What

In [90]:
dataset

Dataset({
    features: ['question', 'answer', 'reference'],
    num_rows: 126
})

### 使用RAGAs 进行评估

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from ragas import evaluate
from ragas.metrics._bleu_score import BleuScore
from ragas.metrics._rouge_score import RougeScore
from ragas.run_config import RunConfig
my_run_config = RunConfig(max_workers=512, timeout=60)
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    answer_similarity,
    context_recall,
    context_precision,
)

embeddings=OpenAIEmbeddings(

)

llm=ChatOpenAI(

)
bleu_score=BleuScore()
rougeL_recall_score = RougeScore(rouge_type="rougeL", measure_type="recall")
rougeL_precision_score = RougeScore(rouge_type="rougeL", measure_type="precision")
rougeL_fmeasure_score = RougeScore(rouge_type="rougeL", measure_type="fmeasure")

result_rougeL_recall_score = evaluate(
    dataset = dataset, 
    metrics=[
        answer_relevancy,
        answer_correctness,
        answer_similarity,
        bleu_score,
        rougeL_recall_score
],
    llm=llm,
    embeddings=embeddings
)

result_rougeL_precision_score = evaluate(
    dataset = dataset, 
    metrics=[
        rougeL_precision_score,
],
    llm=llm,
    embeddings=embeddings
)

result_rougeL_fmeasure_score = evaluate(
    dataset = dataset, 
    metrics=[
        rougeL_fmeasure_score,
],
    llm=llm,
    embeddings=embeddings
)

Evaluating: 100%|██████████| 126/126 [00:01<00:00, 122.91it/s]


In [92]:
print( "rougeL_fmeasure_score=" + str(result_rougeL_recall_score))
print( "rougeL_precision_score=" + str(result_rougeL_precision_score))
print( "rougeL_recall_score=" + str(result_rougeL_fmeasure_score))

rougeL_fmeasure_score={'answer_relevancy': 0.9219, 'answer_correctness': 0.2392, 'semantic_similarity': 0.7946, 'bleu_score': 0.0262, 'rouge_score': 0.2109}
rougeL_precision_score={'rouge_score': 0.0940}
rougeL_recall_score={'rouge_score': 0.1148}
